<a href="https://colab.research.google.com/github/Tavo826/DataScience/blob/main/TrafficSignsRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se pretende entrenar un modelo que clasifique señales de tránsito

### Dataset

Contiene más de 50000 imágenes de diferentes señales de tráfico que se clasifican en 43 clases diferentes, el conjunto de datos está desbalanceado, lo que afecta el rendimietno de la red

### Preprocesamiento

Se itera sobre todas las cases y se agregan las imágenes y las respectivas etiquetas en la lista de data y labels

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Science/Reconocimiento de señales de tránsito')

In [ ]:
data = []
labels = []
current_path = os.getcwd()
classes = len(os.listdir(current_path + '/Train'))

#Recorre las carpetas con las clases
for i in range(classes):
  path = os.path.join(current_path, 'Train', str(i))
  images = os.listdir(path)

  #Modificando las imágenes y agregando a la lista
  for a in images:
    try:
      image = Image.open(path + '/' + a)
      image = image.resize((30,30))
      image = np.array(image)
      data.append(image)
      labels.append(i)
    except:
      print('Error cargando la imagen')
      break

data = np.array(data)
labels = np.array(labels)

El tamaño del conjunto de datos es (39209,30,30,3) los que significa que hay 39209 imágenes de tamaño 30x30 con 3 canales (RGB)

In [ ]:
print('Data shape', data.shape)

Se separa el conjunto en entrenamiento y testeo, luego se convierten las etiquetas a one-hot encoding

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print('Train: ', X_train.shape)
print('Test: ', X_test.shape)

y_train = to_categorical(y_train, classes)
y_test = to_categorical(y_test, classes)

### Construyendo el modelo

Arquitectura de la CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_sape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrenando y validando el modelo

El modelo funciona con un batch size de 64 y su precisión se estabiliza después de 15 épocas

In [ ]:
batch_size = 54
epochs = 15
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_test,, y_test))

### Graficando

Se analizan el accuracy y el loss en el entrenamiento y la validación

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

### Probando el modelo

Se extrae la dirección de la imagen y la etiqueta y se escalan las imágenes para ingresarlas al modelo

In [ ]:
import pandas as pd

y_test = pd.read_csv('Test.csv')

lables = y_test['ClassId'].values
imgs = y_test['Path'].values

data = []

for img in imgs:
  image = Image.open(img)
  image = image.resize((30,30))
  data.append(np.array(image))

X_test = np.array(data)

pred = model.predic_classes(X_test)

accuracy_score(labels, pred)